# GR5293 - Proj1 - Group9
## Classification on the late payment of credit cards
#### Data cleaning pipeline 
* Drop rows with missing rate >= 30%
* After dropping rows, dropping columns with missing rate >= 10%
* Onehot Encoding
* Rule out outliers
* Implement imputations on missing data via KNNimputer
* Datatype conversion
* Drop columns with "DOCUMENTS"


#### Setup

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.impute import KNNImputer
import matplotlib
from matplotlib import pyplot as plt
import time
import os
import re
import gc
gc.enable()
print(os.getcwd())
mydir = os.getcwd() + "/"
%xmode plain
%matplotlib inline

/Users/kangshuoli/Documents/VScode_workspace/GR5293/EODS-Project1-Group9/doc
Exception reporting mode: Plain


#### Read in data

In [2]:
raw_df = pd.read_csv(
    '../data/application_data.csv', 
    header = 0
)
raw_df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


#### Split feature and labels

In [3]:
feature_df = raw_df.iloc[:,2:]
label_df = raw_df.loc[:,"TARGET"]

In [4]:
feature_df

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,Unaccompanied,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,319500.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
label_df.astype(int)

0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64

#### Drop rows with missing rate >= 30%

In [9]:
drop_seq = ["row", "col"]

def cal_missing_rate(df_, axis_):
    drop_seq = ["row", "col"]
    missing_rate_dict = {}
    if axis_ == 0:
        l = df_.shape[1]
        for i in range(df_.shape[axis_]):
            missing_rate_dict[i] = float(df_.iloc[i,:].isna().sum() / l)
    elif axis_ == 1:
        l = df_.shape[0]
        for name in df_.columns:
            missing_rate_dict[name] = float(df_.loc[:,name].isna().sum() / l)    
    return missing_rate_dict

missing_rate_dict_row = cal_missing_rate(df_ = feature_df, axis_ = 0)
# for key, value in missing_rate_dict_row.items():
#     print(key, value)

In [10]:
def drop_with_missing_rate(df_, axis_, threshold_, dict_):
    to_be_dropped = []
    for key, value in dict_.items():
        if value >= threshold_:
            to_be_dropped.append(key)
    new_df = df_.drop(
        to_be_dropped, 
        axis = axis_
    )
    return new_df

data_cleaned_df_row = drop_with_missing_rate(
    df_ = feature_df, 
    axis_ = 0, 
    threshold_ = 0.3, 
    dict_ = missing_rate_dict_row
)

In [12]:
data_cleaned_df_row.index = np.arange(data_cleaned_df_row.shape[0], dtype = int)

In [13]:
data_cleaned_df_row

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Cash loans,F,N,Y,0,67500.0,80865.0,5881.5,67500.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
3,Cash loans,M,Y,N,1,225000.0,918468.0,28966.5,697500.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,Cash loans,F,N,Y,0,189000.0,773680.5,32778.0,679500.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153241,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
153242,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
153243,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,Unaccompanied,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
153244,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,319500.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


#### After dropping rows, dropping cols with missing rate >= 10%

In [14]:
missing_rate_dict_col = cal_missing_rate(df_ = data_cleaned_df_row, axis_ = 1)
data_cleaned_df = drop_with_missing_rate(
    df_ = data_cleaned_df_row, 
    axis_ = 1, 
    threshold_ = 0.1, 
    dict_ = missing_rate_dict_col
)

In [15]:
data_cleaned_df

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0,0,0,0,0,0
2,Cash loans,F,N,Y,0,67500.0,80865.0,5881.5,67500.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0
3,Cash loans,M,Y,N,1,225000.0,918468.0,28966.5,697500.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0
4,Cash loans,F,N,Y,0,189000.0,773680.5,32778.0,679500.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153241,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0
153242,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0
153243,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0
153244,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,319500.0,Unaccompanied,...,0,0,0,0,0,0,0,0,0,0


#### Onehot encoding

In [16]:
data_df_onehot = pd.get_dummies(
    data = data_cleaned_df, 
    drop_first = True
)

In [17]:
data_df_onehot

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,ORGANIZATION_TYPE_XNA,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,0,0,0,0,0,0,0,1,0,0
1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,...,0,0,0,0,0,0,0,0,0,0
2,0,67500.0,80865.0,5881.5,67500.0,0.031329,-13439,-2717,-311.0,-3227,...,0,0,0,0,0,0,0,0,0,0
3,1,225000.0,918468.0,28966.5,697500.0,0.016612,-14086,-3028,-643.0,-4911,...,0,0,0,0,0,0,1,0,0,0
4,0,189000.0,773680.5,32778.0,679500.0,0.010006,-14583,-203,-615.0,-2056,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153241,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,-1982,...,0,0,0,0,0,0,0,1,0,0
153242,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,-4090,...,1,0,0,0,0,0,0,1,0,0
153243,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,-5150,...,0,0,0,0,0,0,1,0,0,0
153244,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,-931,...,0,0,0,0,0,0,0,1,0,0


#### Screen Outlier

#### Imputation

In [18]:
for name in data_df_onehot.columns:
    print(f'{name}: {data_df_onehot[name].isna().sum() / data_df_onehot.shape[0]:0.4f}')

CNT_CHILDREN: 0.0000
AMT_INCOME_TOTAL: 0.0000
AMT_CREDIT: 0.0000
AMT_ANNUITY: 0.0000
AMT_GOODS_PRICE: 0.0008
REGION_POPULATION_RELATIVE: 0.0000
DAYS_BIRTH: 0.0000
DAYS_EMPLOYED: 0.0000
DAYS_REGISTRATION: 0.0000
DAYS_ID_PUBLISH: 0.0000
FLAG_MOBIL: 0.0000
FLAG_EMP_PHONE: 0.0000
FLAG_WORK_PHONE: 0.0000
FLAG_CONT_MOBILE: 0.0000
FLAG_PHONE: 0.0000
FLAG_EMAIL: 0.0000
CNT_FAM_MEMBERS: 0.0000
REGION_RATING_CLIENT: 0.0000
REGION_RATING_CLIENT_W_CITY: 0.0000
HOUR_APPR_PROCESS_START: 0.0000
REG_REGION_NOT_LIVE_REGION: 0.0000
REG_REGION_NOT_WORK_REGION: 0.0000
LIVE_REGION_NOT_WORK_REGION: 0.0000
REG_CITY_NOT_LIVE_CITY: 0.0000
REG_CITY_NOT_WORK_CITY: 0.0000
LIVE_CITY_NOT_WORK_CITY: 0.0000
EXT_SOURCE_2: 0.0020
APARTMENTS_AVG: 0.0214
YEARS_BEGINEXPLUATATION_AVG: 0.0042
ELEVATORS_AVG: 0.0662
ENTRANCES_AVG: 0.0106
FLOORSMAX_AVG: 0.0027
LIVINGAREA_AVG: 0.0240
APARTMENTS_MODE: 0.0214
YEARS_BEGINEXPLUATATION_MODE: 0.0042
ELEVATORS_MODE: 0.0662
ENTRANCES_MODE: 0.0106
FLOORSMAX_MODE: 0.0027
LIVINGAREA_MODE:

In [19]:
imputer = KNNImputer(
    n_neighbors = 5, 
    missing_values = np.nan
)
data_df_imputed = imputer.fit_transform(data_df_onehot)

In [21]:
data_df_imputed = pd.DataFrame(data = data_df_imputed, columns = data_df_onehot.columns)
data_df_imputed.isna().sum().sum()

0

In [22]:
data_df_imputed

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,ORGANIZATION_TYPE_XNA,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,-2120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,-291.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,67500.0,80865.0,5881.5,67500.0,0.031329,-13439.0,-2717.0,-311.0,-3227.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,225000.0,918468.0,28966.5,697500.0,0.016612,-14086.0,-3028.0,-643.0,-4911.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,189000.0,773680.5,32778.0,679500.0,0.010006,-14583.0,-203.0,-615.0,-2056.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153241,0.0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327.0,-236.0,-8456.0,-1982.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
153242,0.0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775.0,365243.0,-4388.0,-4090.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
153243,0.0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966.0,-7921.0,-6737.0,-5150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
153244,0.0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961.0,-4786.0,-2562.0,-931.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
